# Modeling

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from src.static import DATA_DIR

In [2]:
df = pd.read_csv(f'{DATA_DIR}/preprocessed_data.csv')

In [3]:
df.dtypes

year                                             int64
record_number                                    int64
census_tract_2020                                int64
tract_income_ratio                               int64
affordability_cat                                int64
num_units                                      float64
num_bedrooms_0-1                               float64
num_bedrooms_>=2                               float64
affordability_level_>100%                      float64
affordability_level_>50, <=60%                 float64
affordability_level_>60, <=80%                 float64
affordability_level_>80, <=100%                float64
affordability_level_>=0, <=50%                 float64
enterprise_flag_fannie                            bool
enterprise_flag_freddie                           bool
date_of_mortgage_note_prior to year aquired       bool
date_of_mortgage_note_same year as acquired       bool
purpose_of_loan_improvement/rehab                 bool
purpose_of

## Train Test Split
The next step will perform out train test split by finding optimal number of years toward the end of the dataset to get an approximate 80/20 split. The cumulative values below are observed and compared to the actual calculation of `# of rows * 0.80

In [4]:
df.year.value_counts().sort_index().cumsum()

year
2019     7284
2020    15295
2021    22764
2022    29742
2023    34268
Name: count, dtype: int64

In [5]:
22764 / df.shape[0]

0.6642932181627174

In [6]:
(29742 - 22764) / df.shape[0]

0.20363020894128633

Based on the ovserved values we will use years occuring after 2021 as the test set and the previously occuring records as the training set to mimic real world conditions where future information is not allowed into the training data

In [7]:
df = df[[x for x in df.columns if not x.startswith('affordability')]].set_index('record_number')

In [8]:
X_train, X, y_train, y = (df[df.year <= 2021].drop(['num_affordable_units'], axis=1),
               df[df.year > 2021].drop(['num_affordable_units'], axis=1),
               df[df.year <= 2021].num_affordable_units,
               df[df.year > 2021].num_affordable_units)

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_scaled = pd.DataFrame(scaler.transform(X), columns=X.columns)

In [12]:
X_val_scaled, X_test_scaled, y_val, y_test = train_test_split(
    X_scaled, y, test_size = 0.5, stratify=X['year'], random_state=42
    )

In [13]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

# Linear Regression

In [14]:
X_train_scaled

,year,census_tract_2020,tract_income_ratio,num_units,num_bedrooms_0-1,num_bedrooms_>=2,enterprise_flag_fannie,enterprise_flag_freddie,date_of_mortgage_note_prior to year aquired,date_of_mortgage_note_same year as acquired,...,federal_guarantee_FHA,federal_guarantee_no,federal_guarantee_yes,tenant_income_ind_No,tot_num_units_100-149,tot_num_units_25-50,tot_num_units_5-24,tot_num_units_51-99,tot_num_units_> 149,after_covid_ind
0,-1.252338,0.643059,-0.927090,-0.469548,-0.407065,-0.411031,1.050935,-1.050935,2.838749,-2.838749,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,2.356012,-0.455885,-0.456518,-0.769126,-1.457809
1,-1.252338,0.643059,0.388392,-0.519599,-0.468709,-0.438547,1.050935,-1.050935,2.838749,-2.838749,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,2.356012,-0.455885,-0.456518,-0.769126,-1.457809
2,-1.252338,-2.765228,1.703873,-0.469548,-0.407065,-0.411031,1.050935,-1.050935,2.838749,-2.838749,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,-0.424446,-0.455885,2.190495,-0.769126,-1.457809
3,-1.252338,0.643059,0.388392,-0.519599,-0.407065,-0.493580,1.050935,-1.050935,2.838749,-2.838749,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,2.356012,-0.455885,-0.456518,-0.769126,-1.457809
4,-1.252338,0.643059,0.388392,-0.486232,-0.407065,-0.438547,1.050935,-1.050935,2.838749,-2.838749,...,-0.009374,0.045968,-0.044998,0.0,2.574127,-0.424446,-0.455885,-0.456518,-0.769126,-1.457809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22759,1.232147,-1.061084,0.388392,-0.119196,-0.068020,-0.135868,-0.951533,0.951533,-0.352268,0.352268,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,-0.424446,-0.455885,2.190495,-0.769126,0.685961
22760,1.232147,0.643059,0.388392,2.950557,-0.283776,5.119751,-0.951533,0.951533,2.838749,-2.838749,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,-0.424446,-0.455885,-0.456518,1.300178,0.685961
22761,1.232147,0.643059,-0.927090,1.532465,0.640890,1.955373,-0.951533,0.951533,-0.352268,0.352268,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,-0.424446,-0.455885,-0.456518,1.300178,0.685961
22762,1.232147,0.643059,-0.927090,0.247840,1.010757,-0.493580,-0.951533,0.951533,-0.352268,0.352268,...,-0.009374,0.045968,-0.044998,0.0,-0.388481,-0.424446,-0.455885,-0.456518,1.300178,0.685961


In [15]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_val_scaled)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 291.1354331313532
R-squared Score: 0.8732467276971948


In [16]:
[x for x in zip(X_train_scaled.columns, model.coef_)]

[('year', -0.9056524430670176),
 ('census_tract_2020', -1.1329602938559324),
 ('tract_income_ratio', -3.771382908452733),
 ('num_units', -55566541108052.81),
 ('num_bedrooms_0-1', 30077053430527.125),
 ('num_bedrooms_>=2', 33690587004031.51),
 ('enterprise_flag_fannie', 1980650709616.4368),
 ('enterprise_flag_freddie', 1980650709614.9607),
 ('date_of_mortgage_note_prior to year aquired', -7561446871823.371),
 ('date_of_mortgage_note_same year as acquired', -7561446871824.054),
 ('purpose_of_loan_improvement/rehab', 996121310702.5067),
 ('purpose_of_loan_new build', 2916318518308.5557),
 ('purpose_of_loan_purchase', 22689803854676.754),
 ('purpose_of_loan_refinance', 22751381734166.027),
 ('type_of_seller_Other', -3031070505162.4575),
 ('type_of_seller_SAIF or BIF', -2677890425216.296),
 ('type_of_seller_mortgage_company', -3877534716380.1978),
 ('federal_guarantee_FHA', -1854811777589.8208),
 ('federal_guarantee_no', -9077498509020.697),
 ('federal_guarantee_yes', -8886763057856.006),


The massive coefficients on the naive linear regression indicate that there is multicolinearity throwing off our coefficient sizing.

# Ridge Regression
Ridge regression should address some of the colinearity

In [17]:
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0]
}

grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val_scaled)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 290.84049385887323
R-squared Score: 0.8733751370684464


In [18]:
[x for x in zip(X_val_scaled.columns, best_model.coef_)]

[('year', -0.8645432136097008),
 ('census_tract_2020', -1.1255365489331142),
 ('tract_income_ratio', -3.741353217307788),
 ('num_units', 19.59160555225531),
 ('num_bedrooms_0-1', 14.423261910843948),
 ('num_bedrooms_>=2', 19.436542796959046),
 ('enterprise_flag_fannie', 0.7069902795148237),
 ('enterprise_flag_freddie', -0.7069902795148137),
 ('date_of_mortgage_note_prior to year aquired', 0.3329862961493845),
 ('date_of_mortgage_note_same year as acquired', -0.3329862961493925),
 ('purpose_of_loan_improvement/rehab', 0.28638175942584854),
 ('purpose_of_loan_new build', 0.5611856263809352),
 ('purpose_of_loan_purchase', 0.14524982980544923),
 ('purpose_of_loan_refinance', -0.22932924325732548),
 ('type_of_seller_Other', 0.5263627244612102),
 ('type_of_seller_SAIF or BIF', -0.07842538503848145),
 ('type_of_seller_mortgage_company', -0.35729607670218994),
 ('federal_guarantee_FHA', 0.09789267485874137),
 ('federal_guarantee_no', 0.5858527106416641),
 ('federal_guarantee_yes', -0.618858583

In [19]:
best_model

Ridge(alpha=100.0)

# Lasso Regression

In [20]:
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0]
}

grid_search = GridSearchCV(estimator=Lasso(), param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val_scaled)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 280.32869126567056
R-squared Score: 0.8779517197336295


In [21]:
[x for x in zip(X_val_scaled.columns, best_model.coef_)]

[('year', -0.13393859851160445),
 ('census_tract_2020', -0.9724045140137539),
 ('tract_income_ratio', -3.600021930296281),
 ('num_units', 46.137741841184415),
 ('num_bedrooms_0-1', 0.0),
 ('num_bedrooms_>=2', 3.2874789953261083),
 ('enterprise_flag_fannie', 1.2847665928238439),
 ('enterprise_flag_freddie', -0.0),
 ('date_of_mortgage_note_prior to year aquired', 0.5405897960983472),
 ('date_of_mortgage_note_same year as acquired', -2.3971921501658617e-16),
 ('purpose_of_loan_improvement/rehab', 0.18737018418161244),
 ('purpose_of_loan_new build', 0.4471232850844011),
 ('purpose_of_loan_purchase', 0.0),
 ('purpose_of_loan_refinance', -0.25224635753881575),
 ('type_of_seller_Other', 0.5520221399217338),
 ('type_of_seller_SAIF or BIF', 0.0),
 ('type_of_seller_mortgage_company', -0.1920907894691538),
 ('federal_guarantee_FHA', 0.0),
 ('federal_guarantee_no', 0.11797524462463237),
 ('federal_guarantee_yes', -0.9797967840340149),
 ('tenant_income_ind_No', 0.0),
 ('tot_num_units_100-149', 0.30

In [22]:
best_model

Lasso(alpha=0.1)

# Elastic Net

In [23]:
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'l1_ratio': [0.1, 0.5, 0.7, 1.0],
    'max_iter': [1000, 5000, 10000],
    'tol': [1e-4, 1e-5, 1e-6]
}

grid_search = GridSearchCV(estimator=ElasticNet(), param_grid=param_grid, cv=5, scoring='r2',
                            n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val_scaled)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 280.3239865773513
R-squared Score: 0.8779537680402655


In [24]:
[x for x in zip(X_val_scaled.columns, best_model.coef_)]

[('year', -0.1334383958059762),
 ('census_tract_2020', -0.9723214802195531),
 ('tract_income_ratio', -3.5999020737873404),
 ('num_units', 46.137697455258625),
 ('num_bedrooms_0-1', 0.0),
 ('num_bedrooms_>=2', 3.2875722661191067),
 ('enterprise_flag_fannie', 1.284600491731855),
 ('enterprise_flag_freddie', -1.1985960750829313e-16),
 ('date_of_mortgage_note_prior to year aquired', 0.5406300102727277),
 ('date_of_mortgage_note_same year as acquired', -0.0),
 ('purpose_of_loan_improvement/rehab', 0.18738680505832267),
 ('purpose_of_loan_new build', 0.4471792871402317),
 ('purpose_of_loan_purchase', 0.0),
 ('purpose_of_loan_refinance', -0.2519485106581849),
 ('type_of_seller_Other', 0.5524582512022335),
 ('type_of_seller_SAIF or BIF', 0.0),
 ('type_of_seller_mortgage_company', -0.1915816591168441),
 ('federal_guarantee_FHA', -0.0),
 ('federal_guarantee_no', 0.026241118865728427),
 ('federal_guarantee_yes', -1.0696045317253264),
 ('tenant_income_ind_No', 0.0),
 ('tot_num_units_100-149', 0.30

In [25]:
best_model

ElasticNet(alpha=0.1, l1_ratio=1.0, tol=1e-05)

# XGBoost

In [26]:
from xgboost import XGBRegressor

In [ ]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.1, 1.0],
    'reg_lambda': [0.1, 1.0, 10.0]
}
grid_search = GridSearchCV(
    estimator=XGBRegressor(
        objective='reg:squarederror',
        random_state=42
        ),
        param_grid=param_grid,
        cv=5, scoring='r2', n_jobs=-1
    )

grid_search.fit(X_train_scaled, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val_scaled)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")
print(f"Best Parameters: {grid_search.best_params_}")

KeyboardInterrupt: 

## EDA
### Correlation Testing

In [ ]:
train.dtypes

In [ ]:
def get_spearman_corr(df, target):
    correlations = dict()
    for col in df.drop(target, axis=1).select_dtypes(include=np.number).columns:
        corr, p_value = spearmanr(df[col], df[target])
        correlations[col] = {'spearman_corr': corr, 'p_value': p_value}
    correlation_df = pd.DataFrame(correlations).T
    return correlation_df


In [ ]:
corr_df = get_spearman_corr(train, 'affordability_cat')
corr_df[corr_df['p_value'] < 0.05].sort_values(by='spearman_corr')

We have a number of statistically significant correlations with the key takeaways below:
- As affordabliltiy category increases (less affordable) the number more affordabile units decreases
- The number of overall units also decreses
- The tract income ratio only has a small positive corelation with an increase in affordablity category
- Unsuprsingly as affordability category increases the nubmer of less affordable units increases.

These observations cement our understanding of the interplay between affordability_cat and other variables.The most interesting observation here is that despite their apparent implied relationship the tract_income_ratio only has a very weakly positive correlation with the affordability category suggesting that greater incomes do not go hand in hand with lesser affordability.

In [ ]:
corr_df = get_spearman_corr(train, 'census_tract_2020')
corr_df[corr_df['p_value'] < 0.05]

add analysis here

In [ ]:
corr_df = get_spearman_corr(train, 'tract_income_ratio')
corr_df[corr_df['p_value'] < 0.05]

add analysis here